In [ ]:
import pandas as pd
import re
import string
import numpy as np
from typing import Dict, List, Tuple, Optional

# 前提：已经分配好了车辆型号给文本
# --------------------------
# 1. 预统计位置规律（只需运行一次）
# --------------------------
def 统计位置规律(标注数据路径: str) -> Dict[str, Dict]:
    """从标注数据中统计每个型号的年份位置规律"""
    数据 = pd.read_excel(标注数据路径)
    规律 = {}  # {型号: {"前置概率": x, "搜索范围": y}, ...}
    
    for _, 行 in 数据.iterrows():
        文本 = str(行['文本'])
        型号列表 = [i.strip().lower() for i in str(行['型号']).split(',')]
        年份列表 = [i.strip() for i in str(行['年份范围']).split(',')]
        
        for 型号, 年份 in zip(型号列表, 年份列表):
            型号位置 = 文本.lower().find(型号)
            年份位置 = 文本.find(年份)
            if 型号位置 == -1 or 年份位置 == -1:
                continue
            
            距离 = 年份位置 - 型号位置
            方向 = "前置" if 距离 < 0 else "后置"
            
            if 型号 not in 规律:
                规律[型号] = {"距离列表": [], "前置次数": 0, "总次数": 0}
            
            规律[型号]["距离列表"].append(abs(距离))
            规律[型号]["总次数"] += 1
            if 方向 == "前置":
                规律[型号]["前置次数"] += 1
    
    # 计算最终规律
    for 型号 in 规律:
        距离列表 = 规律[型号]["距离列表"]
        总次数 = 规律[型号]["总次数"]
        前置概率 = 规律[型号]["前置次数"] / 总次数 if 总次数 > 0 else 0.5
        分位数 = np.percentile(距离列表, 90) if 距离列表 else 50
        规律[型号] = {
            "前置概率": 前置概率,
            "搜索范围": int(分位数) + 5  # 加缓冲
        }
    print(规律)
    return 规律

# --------------------------
# 2. 核心函数：指定型号提取年份
# --------------------------
def 提取指定型号的年份(
    文本: str,
    目标型号: str,
    位置规律: Dict[str, Dict],
    默认搜索范围: int = 50  # 无规律时的默认范围
) -> Optional[str]:
    """
    从文本中提取指定型号对应的年份
    参数：
        文本: 待处理的配件描述文本
        目标型号: 用户指定的型号（如"silverado"）
        位置规律: 预统计的位置规律字典
        默认搜索范围: 无规律时的默认搜索范围（字符数）
    返回：
        提取到的年份范围（如"2001-2019"），未找到则返回None
    """
    # 标准化目标型号（统一小写，去除标点）
    目标型号标准化 = 目标型号.translate(
        str.maketrans("", "", string.punctuation)
    ).lower().strip()
    文本标准化 = 文本.lower()
    原始文本 = 文本  # 保留原始文本用于提取年份
    
    # 步骤1：在文本中定位目标型号的位置
    型号长度 = len(目标型号标准化)
    文本长度 = len(文本标准化)
    型号位置列表 = []  # 存储所有匹配的位置 (起始索引, 结束索引)
    
    for i in range(文本长度 - 型号长度 + 1):
        子串 = 文本标准化[i:i+型号长度]
        # 简单匹配（可根据需要替换为之前的字符串相似度计算）
        if 子串 == 目标型号标准化:
            型号位置列表.append((i, i + 型号长度))
    
    if not 型号位置列表:
        print(f"未在文本中找到型号：{目标型号}")
        return None
    
    # 步骤2：获取该型号的位置规律
    型号规律 = 位置规律.get(目标型号标准化, {
        "前置概率": 0.5,
        "搜索范围": 默认搜索范围
    })
    搜索范围 = 型号规律["搜索范围"]
    
    # 步骤3：在每个匹配的型号位置附近搜索年份
    for (型号起始, 型号结束) in 型号位置列表:
        # 划定搜索范围：型号前后[搜索范围]字符内
        文本总长度 = len(原始文本)
        范围起始 = max(0, 型号起始 - 搜索范围)
        范围结束 = min(文本总长度, 型号结束 + 搜索范围)
        搜索区域 = 原始文本[范围起始:范围结束]
        
        # 用正则提取年份（支持####-####和####-##格式）
        年份匹配 = re.search(r'\b\d{4}-\d{2,4}\b', 搜索区域)
        if 年份匹配:
            年份字符串 = 年份匹配.group()
            # 补全年份简写（如2010-20 → 2010-2020）
            年份部分 = 年份字符串.split('-')
            if len(年份部分[1]) == 2:
                try:
                    if int(年份部分[0]) >= 2000:
                        年份部分[1] = f"20{年份部分[1]}"
                    else:
                        年份部分[1] = f"19{年份部分[1]}"
                    年份字符串 = f"{年份部分[0]}-{年份部分[1]}"
                except:
                    pass  # 格式异常则不处理
            return 年份字符串
    
    # 所有位置都未找到年份
    print(f"未在{目标型号}附近找到年份")
    return None

# --------------------------
# 示例使用
# --------------------------
if __name__ == "__main__":
    # 1. 预统计规律（实际使用时只需运行一次，可保存为文件复用）
    标注数据路径 = "C:/Users/Administrator/Desktop/车型标注数据.xlsx"  # 按之前的格式准备
    位置规律 = 统计位置规律(标注数据路径)
    
    # 2. 测试提取指定型号的年份
    测试文本 = """
    1. Spare Tire Tool Kit for Silverado 2001-2019, compatible with Tahoe 2010-2020
    2. 2015-2019 Yukon XL配件，适用于1999-2019 Silverado
    3. Brake pads fit 2005-2015 Sierra and 2010-2020 Tahoe
    """
    # 用户指定型号，提取对应年份
    目标型号1 = "silverado"
    年份1 = 提取指定型号的年份(测试文本, 目标型号1, 位置规律)
    print(f"型号 {目标型号1} 的年份：{年份1}")  # 预期：2001-2019 或 1999-2019

    temp = []
    df = pd.read_excel("C:/Users/Administrator/Desktop/车型标注数据.xlsx" )
    for i,j in zip(df['文本'],df['型号']):
        年份 = 提取指定型号的年份(i, j, 位置规律)
        temp.append(年份)
    a = 0
    for i in range(len(df['年份范围'].to_list())):
        if df['年份范围'].to_list()[i] == temp[i]:
            a = a + 1
        else:
            print(df['年份范围'].to_list()[i],temp[i])
    print(df.shape[0])
    print(a/df.shape[0])

{'silverado': {'前置概率': 0.36363636363636365, '搜索范围': 40}, 'tahoe': {'前置概率': 0.5, '搜索范围': 41}, 'avalanche': {'前置概率': 0.5, '搜索范围': 38}, 'suburban': {'前置概率': 0.5, '搜索范围': 41}, 'sierra': {'前置概率': 0.3333333333333333, '搜索范围': 48}, 'yukon': {'前置概率': 0.75, '搜索范围': 57}, 'escalade': {'前置概率': 0.6, '搜索范围': 71}, 'f-150': {'前置概率': 0.6, '搜索范围': 25}, 'tacoma': {'前置概率': 0.5, '搜索范围': 24}, 'ram 1500': {'前置概率': 0.7692307692307693, '搜索范围': 21}, 'ram 1500 classic': {'前置概率': 1.0, '搜索范围': 15}, 'gladiator': {'前置概率': 1.0, '搜索范围': 60}, 'frontier': {'前置概率': 0.6666666666666666, '搜索范围': 30}, 'xterra': {'前置概率': 0.5, '搜索范围': 22}, 'tundra': {'前置概率': 0.6666666666666666, '搜索范围': 20}, 'sequoia': {'前置概率': 1.0, '搜索范围': 15}, 'pathfinder': {'前置概率': 1.0, '搜索范围': 39}, 'titan': {'前置概率': 1.0, '搜索范围': 36}, 'e-150': {'前置概率': 0.0, '搜索范围': 23}, 'e-250': {'前置概率': 0.0, '搜索范围': 17}, 'e-350': {'前置概率': 0.0, '搜索范围': 11}, 'e-450 super duty': {'前置概率': 1.0, '搜索范围': 15}, 'econoline super duty': {'前置概率': 1.0, '搜索范围': 15}, 'durango': {'前置概率': 0.